In [1]:
!pip install -q torchtoolbox timm

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import os
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from sklearn import model_selection
import torchvision.transforms as transforms
import torchvision.io 
import librosa
from PIL import Image
import albumentations as alb
import torch.multiprocessing as mp
import warnings
from torchtoolbox.tools import mixup_data, mixup_criterion
import torch.nn as nn
from torch.nn.functional import cross_entropy
import torchmetrics
import timm
import sklearn.metrics

warnings.filterwarnings('ignore')

In [3]:
class Config:
    num_classes = 264
    batch_size = 64  
    model = "skresnet18"
    pretrained = True   
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    

    data_root = "/kaggle/input/birdclef-2023/"
    train_images = "/kaggle/input/split-creating-melspecs-stage-1/specs/train/"
    valid_images = "/kaggle/input/split-creating-melspecs-stage-1/specs/valid/"
    train_path = "/kaggle/input/bc2023-train-val-df/train.csv"
    valid_path = "/kaggle/input/bc2023-train-val-df/valid.csv"
    

    SR = 32000
    DURATION = 5
    MAX_READ_SAMPLES = 5    

In [4]:
df_train = pd.read_csv(Config.train_path)
df_train = pd.concat([df_train, pd.get_dummies(df_train['primary_label'])], axis=1)

df_valid = pd.read_csv(Config.valid_path)
df_valid = pd.concat([df_valid, pd.get_dummies(df_valid['primary_label'])], axis=1)
birds = list(df_train.primary_label.unique())
missing_birds = list(set(list(df_train.primary_label.unique())).difference(list(df_valid.primary_label.unique())))
non_missing_birds = list(set(list(df_train.primary_label.unique())).difference(missing_birds))
df_valid[missing_birds] = 0
df_valid = df_valid[df_train.columns] 

In [5]:
class BirdDataset(torch.utils.data.Dataset):

    def __init__(self, df, sr = Config.SR, duration = Config.DURATION, augmentations = None, train = True):

        self.df = df
        self.sr = sr 
        self.train = train
        self.duration = duration
        self.augmentations = augmentations
        self.img_dir = Config.valid_images

    def __len__(self):
        return len(self.df)

    @staticmethod
    def normalize(image):
        image = image / 255.0
        return image

    def __getitem__(self, idx):

        row = self.df.iloc[idx]
        impath = self.img_dir + f"{row.filename}.npy"
        image = np.load(str(impath))[:Config.MAX_READ_SAMPLES]
        image = image[0]
        image = torch.tensor(image).float()
        image = torch.stack([image, image, image])
        image = self.normalize(image)

        return image, torch.tensor(row[17:]).float()


In [6]:
def padded_cmap(solution, submission, padding_factor=5):
    solution = solution
    submission = submission
    new_rows = []
    for i in range(padding_factor):
        new_rows.append([1 for i in range(len(solution.columns))])
    new_rows = pd.DataFrame(new_rows)
    new_rows.columns = solution.columns
    padded_solution = pd.concat([solution, new_rows]).reset_index(drop=True).copy()
    padded_submission = pd.concat([submission, new_rows]).reset_index(drop=True).copy()
    score = sklearn.metrics.average_precision_score(
        padded_solution.values,
        padded_submission.values,
        average='macro',
    )
    return score

def map_score(solution, submission):
    solution = solution
    submission = submission
    score = sklearn.metrics.average_precision_score(
        solution.values,
        submission.values,
        average='micro',
    )
    return score

In [7]:
ds_val = BirdDataset(df_valid, sr = Config.SR, duration = Config.DURATION, augmentations = None, train = False)
dl_val = DataLoader(ds_val, batch_size=Config.batch_size, num_workers = 2)

model = timm.create_model(Config.model, pretrained=True)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, Config.num_classes)
model.to(Config.device)
print("Model loaded")

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/skresnet18_ra-4eec2804.pth" to /root/.cache/torch/hub/checkpoints/skresnet18_ra-4eec2804.pth


Model loaded


In [8]:
class BirdClefModel(pl.LightningModule):
    def __init__(self, model_name=Config.model, num_classes = Config.num_classes, pretrained = Config.pretrained):
        super().__init__()
        self.num_classes = num_classes
        self.backbone = timm.create_model(model_name, pretrained=pretrained)
        self.in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(self.in_features, num_classes)
        
        checkpoint = torch.load('/kaggle/input/new-modal-weights/skresnet18_with_contrastive loss.pth')
        self.backbone.load_state_dict(checkpoint['model_state_dict'])
    
    def forward(self,images):
        logits = self.backbone(images)
        return logits
    
    def test_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        return {"logits": outputs, "targets": targets}
    
    def test_epoch_end(self, outputs):
        
        output_val = torch.cat([x['logits'] for x in outputs],dim=0).sigmoid().cpu().detach().numpy()
        target_val = torch.cat([x['targets'] for x in outputs],dim=0).cpu().detach().numpy()
        
        val_df = pd.DataFrame(target_val, columns = birds)
        pred_df = pd.DataFrame(output_val, columns = birds)
        
        avg_score = padded_cmap(val_df, pred_df, padding_factor = 5)
        print(f'C-MAP score :{avg_score}')


In [9]:
model = BirdClefModel()
ds_val = BirdDataset(
        df_valid, 
        sr = Config.SR,
        duration = Config.DURATION,
        augmentations = None,
        train = False
    )
dl_val = DataLoader(ds_val, batch_size=Config.batch_size, num_workers = 2)
    
trainer = pl.Trainer(gpus=1)  

trainer.test(model, dl_val)

Testing: 0it [00:00, ?it/s]

C-MAP score :0.8258541687125813


[{}]